In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install -q pyomo
from pyomo.environ import *

# installing qpsolvers from web
!pip -q install qpsolvers[open_source_solvers]
from qpsolvers import solve_qp
!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
# Install solver as well into collab session
!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc


Looking in indexes: https://pypi.gurobi.com


QUESTION 0

In [ ]:
#APPROACH 1

#total variables
n=5

#cost matrix for u's
c= np.array([1 for i in range(n)])

A = np.array([[85,92,45,27,31],
  [92,54,22,20,7],
  [96,67,29,20,11],
  [-91, -57, -33, -23, -12 ],
  [-99, -75, -26, -24, -41 ],
  [-98, -99, -57, -45, -65 ]])

# then there are constraints on dummy vs original variables
#xi <= ui
#-xi <= ui
#ui >= 0




In [ ]:
model = ConcreteModel()
model.x = Var(np.arange(n), domain=Reals)
model.u = Var(np.arange(n), domain=Reals)

model.obj = Objective(expr = sum(model.u[i]*c[i] for i in range(n)), sense=minimize)
model.constraints = ConstraintList()

for i in range(6):
  model.constraints.add(expr = sum(A[i][j]*model.x[j] for j in range(5))>=1)

for i in range(5):
  model.constraints.add(expr = model.u[i] >= model.x[i])
  model.constraints.add(expr = model.u[i] >= -model.x[i])
  model.constraints.add(expr = model.u[i] >=0)
model.pprint()



3 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   21 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}
    u_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    u : Size=5, Index=u_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :  None :  None :  None : False :  True :  Reals
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals
          3 :  None :  None :  None : False :  True :  Reals
          4 :  None :  None :  None : False :  True :  Reals
    x : Size=5, Index=x_index
        Key

In [ ]:
opt = SolverFactory("cbc")
results = opt.solve(model)

In [ ]:
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.5479557323, 'Upper bound': 0.5479557323, 'Number of objectives': 1, 'Number of constraints': 21, 'Number of variables': 10, 'Number of nonzeros': 5, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 13}}, 'Error rc': 0, 'Time': 0.021881580352783203}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
model.obj()

0.547955725

In [ ]:
for i in range(5):
  print(f"x{i}:", model.x[i].value)

x0: -0.046448575
x1: 0.18612441
x2: -0.17131802
x3: 0.0
x4: -0.14406472


APPROACH 2


In [ ]:
model2 = ConcreteModel()
model2.a = Var(np.arange(n), domain=NonNegativeReals)
model2.b = Var(np.arange(n), domain=NonNegativeReals)

model2.obj = Objective(expr = sum((model2.a[i]+model2.b[i])*c[i] for i in range(n)), sense=minimize)
model2.constraints = ConstraintList()

for i in range(6):
  model2.constraints.add(expr = sum(A[i][j]*(model2.a[j]-model2.b[j]) for j in range(5))>=1)
model2.pprint()


3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    b_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}

2 Var Declarations
    a : Size=5, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    b : Size=5, Index=b_index
        Key : Lowe

In [ ]:
opt2 = SolverFactory('cbc')
results2 = opt2.solve(model2)

In [ ]:
results2

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.5479557323, 'Upper bound': 0.5479557323, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 10, 'Number of nonzeros': 10, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 5}}, 'Error rc': 0, 'Time': 0.02214193344116211}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
model2.obj()

0.547955725

In [ ]:
for i in range(5):
  print(f"a{i}:", model2.a[i].value, end=" | ")
  print(f"b{i}:", model2.b[i].value)

a0: 0.0 | b0: 0.046448575
a1: 0.18612441 | b1: 0.0
a2: 0.0 | b2: 0.17131802
a3: 0.0 | b3: 0.0
a4: 0.0 | b4: 0.14406472


In [ ]:
for i in range(5):
  print(f"x{i} = a{i} - b{i}:", model2.a[i].value-model2.b[i].value)

x0 = a0 - b0: -0.046448575
x1 = a1 - b1: 0.18612441
x2 = a2 - b2: -0.17131802
x3 = a3 - b3: 0.0
x4 = a4 - b4: -0.14406472


In [ ]:
# printing model1 values here to compare
for i in range(5):
  print(f"x{i} = a{i} - b{i}:", model.x[i].value)

x0 = a0 - b0: -0.046448575
x1 = a1 - b1: 0.18612441
x2 = a2 - b2: -0.17131802
x3 = a3 - b3: 0.0
x4 = a4 - b4: -0.14406472


In [37]:
for i in model.constraints:
  print(model.constraints[i]())

0.9999996250000001
0.9999997599999997
1.458337769999999
1.0000002550000002
1.000000215000001
5.2549776999999995
-0.09289715
0.0
0.046448575
0.0
-0.37224882
0.18612441
-0.34263604
0.0
0.17131802
0.0
0.0
0.0
-0.28812944
0.0
0.14406472


In [38]:
for i in model2.constraints:
  print(model2.constraints[i]())

0.9999996250000001
0.9999997599999997
1.458337769999999
1.0000002550000002
1.000000215000001
5.2549776999999995
